In [1]:
import ipyparallel as ipp

cluster = ipp.Cluster.from_file("/root/.ipython/profile_default/security/cluster-.json")
rc = cluster.connect_client_sync()
rc

In [2]:
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import segmentation_models as sm

2022-11-06 07:05:34.534091: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-06 07:05:34.604395: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-06 07:05:34.624298: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Segmentation Models: using `keras` framework.


In [3]:
ROOT_DIR = os.path.abspath(os.getcwd())
DATASETS_DIR = os.path.join(ROOT_DIR, "datasets")
ICH_420_DATASET = os.path.join(DATASETS_DIR, "ICH_420")
ICH_420_DATASET_TFRECORDS = os.path.join(ICH_420_DATASET, "TFRecords")
ICH_420_DATASET_TFRECORDS_TRAIN = os.path.join(ICH_420_DATASET_TFRECORDS, "train")

In [4]:
def split_train_test(tfrecords_pattern, test_rate=0.2, buffer_size=10000):
    filenames = tf.io.gfile.glob(tfrecords_pattern)
    random.shuffle(filenames)
    split_idx = int(test_rate * len(filenames))
    

    return filenames[split_idx:], filenames[:split_idx]

In [5]:
train_dataset_filepaths, test_dataset_filepaths = split_train_test(os.path.join(ICH_420_DATASET_TFRECORDS_TRAIN, "*.tfrecord"))
print(f"Train: {len(train_dataset_filepaths)}")
print(f"Test: {len(test_dataset_filepaths)}")

Train: 605
Test: 151


In [6]:
def decode_image(image, img_type, dtype=tf.float32, bit=12):
    image = tf.io.parse_tensor(image, out_type=np.uint16, name=img_type)
    image = tf.expand_dims(image, axis=-1)
    image = tf.cast(image, dtype) / (2 ** bit)
    
    return image

In [7]:
def read_tfrecord(example):
    features_description = {
        "filename": tf.io.FixedLenFeature([], tf.string),
        "number": tf.io.FixedLenFeature([], tf.int64),
        "sample": tf.io.FixedLenFeature([], tf.int64),
        "image_raw": tf.io.FixedLenFeature([], tf.string),
        "mask_raw": tf.io.FixedLenFeature([], tf.string)
    }
    
    example = tf.io.parse_single_example(example, features_description, name="nii")
    image = decode_image(example["image_raw"], img_type="image")
    mask = decode_image(example["mask_raw"], img_type="mask")

    return image, mask

In [8]:
def load_dataset(filenames):
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.shuffle(2048, reshuffle_each_iteration=False)
    dataset = dataset.map(read_tfrecord, num_parallel_calls=tf.data.AUTOTUNE)
    
    return dataset

In [9]:
def get_dataset(filenames):
    dataset = load_dataset(filenames)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    dataset = dataset.batch(8)
    dataset = dataset.repeat()
    
    return dataset

In [16]:
train_dataset = get_dataset(train_dataset_filepaths)
test_dataset = get_dataset(test_dataset_filepaths)

train, test = iter(train_dataset).next()

In [12]:
preprocess_input = sm.get_preprocessing("resnet50")
train_dataset = preprocess_input(train_dataset)

In [13]:
model = sm.Unet("resnet50", encoder_weights=None, input_shape=(None, None, 1))
# tf.keras.utils.plot_model(model, show_dtype=True)

In [14]:
model.compile(
    'Adam',
    loss=sm.losses.dice_loss,
    metrics=[sm.metrics.iou_score, sm.metrics.f1_score],
)

In [15]:
model.fit(train_dataset,
          epochs=2,
          steps_per_epoch=1000,
          verbose=2
         )

Epoch 1/2


2022-11-06 07:05:50.984599: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:390] Filling up shuffle buffer (this may take a while): 1563 of 2048
2022-11-06 07:05:54.318851: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:415] Shuffle buffer filled.
2022-11-06 07:05:54.838453: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2022-11-06 07:05:55.708555: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


InvalidArgumentError: Graph execution error:

2 root error(s) found.
  (0) INVALID_ARGUMENT:  Type mismatch between parsed tensor (int32) and dtype (uint16)
	 [[{{node image}}]]
	 [[IteratorGetNext]]
	 [[cond_1/then/_39/cond_1/cond/then/_204/cond_1/cond/remove_squeezable_dimensions/Rank_1/_118]]
  (1) INVALID_ARGUMENT:  Type mismatch between parsed tensor (int32) and dtype (uint16)
	 [[{{node image}}]]
	 [[IteratorGetNext]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_14813]